In [1]:
#default_exp data

# Data
> Basic functionalities to read the NYU dataset

In [4]:
#export
import torch
import torchvision.transforms as T
from torch.utils import data

import numpy as np
import matplotlib.pyplot as plt
import mat73
from fastcore.all import *

from hydranet.utils import *

## Dataset

In [5]:
#export
def matlab_to_np(dataset_path = 'data/nyu_depth_v2_labeled.mat'):
    "Save on npy files our dataset"
    data = mat73.loadmat(dataset_path)
    np.save("data/images", data["images"])
    np.save("data/depths", data["depths"])
    np.save("data/labels", data["labels"])

In [8]:
#export
class NYUDataset:
    
    _titles=["img", "depth", "label"]
    
    def __init__(self, dataset_path='data', tfms=None):
        self.images = np.load(Path(dataset_path)/"images.npy")
        self.depths = np.load(Path(dataset_path)/"depths.npy")            
        self.labels = np.load(Path(dataset_path)/"labels.npy")

        
    def __getitem__(self, idx):
        timg = T.ToTensor()(self.images[..., idx])
        tdepth = torch.as_tensor(self.depths[None, :, :, idx])
        tlabel = torch.as_tensor(self.labels[:, :, idx].astype(np.int64))
        return (timg, tdepth, tlabel)
        
    def show_one(self, idx=0):
        show_images(self.__getitem__(idx), titles=self._titles)
    
    def __len__(self): return self.images.shape[-1]

    def dataloader(self, batch_size: int, num_workers: int, shuffle: bool=False, pin_memory: bool=False) -> data.DataLoader:
         return data.DataLoader(
             self,
             batch_size=batch_size,
             num_workers=num_workers,
             shuffle=shuffle,
             pin_memory=pin_memory,
         )

## Export -

In [9]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data.ipynb.
Converted 02_layers.ipynb.
Converted 03_hydranet.ipynb.
Converted 04_trainer.ipynb.
